# convertin a docx into a structured dataframe

this feature engineering task transforms a docx to tidy data.
it can eventually be exported as notebooks.

this work is important because we when open existing course content 
we want to avoid redundant work. spending <1 week to feature engineer
existing content into structured notebooks is a reasonable time line.
developers become familiar with the content through this data engineering activity.: typing.Sequence[docx.Type]

In [336]:
    %pip install docx pandas toolz jinja2
    import docx, itertools, pandas; from toolz.curried import *
    g = compose_left
    PANDOC = shutil.which("pandoc")

    import docx, itertools, pandas; from toolz.curried import *
    g = compose_left
    PANDOC = shutil.which("pandoc")

In [337]:
    document = docx.Document("/home/tbone/Downloads/2015-07-15-Textbook-Cambridge-U-P.docx")

    document = docx.Document("/home/tbone/Downloads/2015-07-15-Textbook-Cambridge-U-P.docx")

In [338]:
the snippet discovers the style id's of each paragraph. this means we could remediate the the headings here and make them formal headings

the snippet discovers the style id's of each paragraph. this means we could remediate the the headings here and make them formal headings

In [339]:
    paragraphs = pandas.Series(document.iter_inner_content())
    df = paragraphs.to_frame("p").join(
        paragraphs.apply(
            g(operator.attrgetter("style"), operator.attrgetter("style_id"))
        ).rename("style_id")
    )
    
{{df.style_id.value_counts().to_frame("word defined style ids").T}}

    end_of_toc = df.style_id.str.match("^TOC").pipe(lambda x: x[x]).index[-1] + 1
    toc, df, refs = df.iloc[:end_of_toc], df.iloc[end_of_toc:], df.iloc[-1:]

    paragraphs = pandas.Series(document.iter_inner_content())
    df = paragraphs.to_frame("p").join(
        paragraphs.apply(
            g(operator.attrgetter("style"), operator.attrgetter("style_id"))
        ).rename("style_id")
    )
    
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>style_id</th>
      <th>Normal</th>
      <th>Caption</th>
      <th>Left</th>
      <th>Heading3</th>
      <th>test</th>
      <th>Heading2</th>
      <th>BodyTextIndent</th>
      <th>ListParagraph</th>
      <th>Heading1</th>
      <th>NormalWeb</th>
      <th>TableNormal</th>
      <th>TableGrid</th>
      <th>BodyTextIndent3</th>
      <th>Heading4</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>word defined style ids</th>
      <td>2202</td>
      <td>438</td>
      <td>343</td>
      <td>47</td>
      <td>47</td>
      <td>38</td>
      <td>35</td>
      <td>21</td>
      <td>10</td>
      <td>6</td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
    </tr>
  </tbody>
</table>
</div>

    end_of_toc = df.style_id.str.match("^TOC").pipe(lambda x: x[x]).index[-1] + 1
    toc, df, refs = df.iloc[:end_of_toc], df.iloc[end_of_toc:], df.iloc[-1:]

In [340]:
    def get_docx_subset(elements: typing.Sequence[docx.Type]) -> docx.document.Document:
        """create a new docx object with a subset of elements from an original"""
        (new := docx.Document())._body._element.extend(x._element for x in elements)
        return new
    
    async def get_docx_subset_markdown(elements: typing.Sequence[docx.Type]) -> str:
        """shell out to pandoc to convert the docx elements to markdown"""
        tmp = tempfile.NamedTemporaryFile(suffix=".docx")
        get_docx_subset(elements).save(tmp.name)
        return subprocess.check_output([PANDOC, "--to", "gfm", "--from", "docx", tmp.name]).decode()

    def get_docx_subset(elements: typing.Sequence[docx.Type]) -> docx.document.Document:
        """create a new docx object with a subset of elements from an original"""
        (new := docx.Document())._body._element.extend(x._element for x in elements)
        return new
    
    async def get_docx_subset_markdown(elements: typing.Sequence[docx.Type]) -> str:
        """shell out to pandoc to convert the docx elements to markdown"""
        tmp = tempfile.NamedTemporaryFile(suffix=".docx")
        get_docx_subset(elements).save(tmp.name)
        return subprocess.check_output([PANDOC, "--to", "gfm", "--from", "docx", tmp.name]).decode()

define chapters by the 10 h1s we found and the nearest heading to identify notebooks.

In [341]:
    notebooks = df.join(
        df[df.style_id.str.match("^Heading1")].p.apply(operator.attrgetter("text")).rename("chapter")
    ).join(
        df[df.style_id.str.match("^Heading")].p.apply(operator.attrgetter("text")).rename("notebook")
    ).ffill().fillna("empty").groupby(["chapter", "notebook"])

    notebooks = df.join(
        df[df.style_id.str.match("^Heading1")].p.apply(operator.attrgetter("text")).rename("chapter")
    ).join(
        df[df.style_id.str.match("^Heading")].p.apply(operator.attrgetter("text")).rename("notebook")
    ).ffill().fillna("empty").groupby(["chapter", "notebook"])

turned the grouped notebooks to markdown

In [342]:
    (md := pandas.Series(
        await asyncio.gather(*(get_docx_subset_markdown(notebooks.get_group(x).p) for x in notebooks.groups)),
        notebooks.groups
    ).to_frame("markdown"))

markdown
Applications to chemical reactions        Applications to chemical reactions                  # Applications to chemical reactions\n\nA chem...
                                          Ellingham diagram and buffered systems              ## Ellingham diagram and buffered systems\n\nO...
                                          Internal process and differential and integrate...  ## Internal process and differential and integ...
                                          Maximum reaction rate and chemical transport re...  ## Maximum reaction rate and chemical transpor...
                                          Trends of entropies of reactions                    ## Trends of entropies of reactions\n\nThe rea...
...                                                                                                                                         ...
Phase equilibria in heterogeneous systems Projection and section of phase diagrams with p...  ### Projection and section of phase diagrams w...
                                          Section of potential phase diagrams                 ### Section of potential phase diagrams\n\nBas...
                                          Tie-lines and lever rule                            ### Tie-lines and lever rule\n\nIt is self-evi...
References                                References                                          # References\n\n1\. M. Hillert, *Phase Equilib...
empty                                     empty                                               Thermodynamics is a science concerning the sta...

[97 rows x 1 columns]

    (md := pandas.Series(
        await asyncio.gather(*(get_docx_subset_markdown(notebooks.get_group(x).p) for x in notebooks.groups)),
        notebooks.groups
    ).to_frame("markdown"))

below we see that most of this transformation renders pretty ok on notebooks

In [343]:
    display(*md[md.markdown.str.contains("\$")].iloc[:10].markdown.apply(Markdown))

## Ellingham diagram and buffered systems

One type of chemical reactions is between a pure element in liquid or
solid states and its oxides involving one mole of oxygen gas, i.e.

*Eq. 7‑7* $\frac{4}{v_{M}}M + O_{2}(gas) = M_{4/v_{M}}O_{2}$

with $v_{M}$ being the valence of the element $M$ in the oxide. Taking
the pure $M$ and the gaseous $O_{2}$ at the reaction temperature and one
atmospheric pressure as their respective reference states, both
activities of the pure $M$ solid or liquid phase and its oxide are
unity, and the activity of $O_{2}$ equals to its partial pressure in an
ideal gas. becomes

*Eq. 7‑8*
$RTlnP_{O_{2}} = \mathrm{\Delta}_{\ }^{0}G = \mathrm{\Delta}_{\ }^{0}H - T\mathrm{\Delta}_{\ }^{0}S$

Based on , one can plot $\mathrm{\Delta}_{\ }^{0}G$ as a function of
temperature for various oxidation reactions, which is called Ellingham
diagram. The intercept at $T = 0K$ is given by
$\mathrm{\Delta}_{\ }^{0}H$, and the slope is represented by
$- \mathrm{\Delta}_{\ }^{0}S$, depicted by the following equation

*Eq. 7‑9*
$\mathrm{\Delta}_{\ }^{0}S = S_{M_{4/v_{M}}O_{2}} - S_{O_{2}} - \frac{4}{v_{M}}S_{M}$

Since the entropy of one mole of $O_{2}$ gas is significantly larger
than those of the pure element and its oxide when they are in solid or
liquid states and the entropy difference between the pure element and
its oxide, the entropy of reaction is thus dominated by the reduction of
the entropy by one mole of $O_{2}$ gas. Consequently, the entropies of
reaction are approximately the same for most reactions when the pure
elements and their oxides are solid as seen in where most lines have
similar slopes.

Figure ‑: Ellingham Diagram for a number of metal-oxide systems.

For a chemical reaction on the Ellingham diagram at a given temperature,
$\mathrm{\Delta}_{\ }^{0}G$ can be read from the y-axis of the diagram,
and the equilibrium partial pressure of $O_{2}$ gas can be calculated
using . Alternatively, one can plot the left part of for a given
$P_{O_{2}}$ as a function of temperature on the Ellingham diagram, i.e.

*Eq. 7‑10* $\mathrm{\Delta}_{\ }^{0}{G =}RTlnP_{O_{2}}$

This results in straight lines, representing iso-partial-pressure lines
of $O_{2}$, with the intercept being zero at $T = 0K$ and slopes of
$RlnP_{O_{2}}$, which are negative for $P_{O_{2}}$ lower than one
atmospheric pressure (1atm). The values of $P_{O_{2}}$ are marked on the
secondary axis on the right of the Ellingham diagram. The intersection
of the isoactivitiy line and the equilibrium line of the chemical
reaction thus gives the relation of equilibrium temperature and
equilibrium partial pressure of $O_{2}$. This is demonstrated in .

Figure ‑: Intersection of iso-partial-pressure lines of $O_{2}$ and
equilibrium lines in the Ellingham diagram

For each chemical reaction in the Ellingham diagram, the three phases
are in equilibrium on the line represented by , i.e. metal, metal
oxides, and O<sub>2</sub> gas. For conditions above the line, the value
of $P_{O_{2}}$ is larger than its equilibrium value, and the metal will
be oxidized. For conditions below the line, the value of $P_{O_{2}}$ is
lower than its equilibrium value, and the metal oxide will be reduced.
Therefore, the metal oxides in the upper part of the Ellingham diagram
can be reduced by the metals in the lower part of the diagram, and vice
versus, metals in the lower part of the diagram can be oxidized by the
metal oxides in the upper part of the diagram. For example, in , Ca can
reduce all oxides, and Cu<sub>2</sub>O is the least stable oxide.

From the above Ellingham diagram, it is noted that equilibrium partial
pressures of $O_{2}$ are very low for most chemical reactions with many
of them lower than $10^{- 12}\ atm$. One approach to obtain such a low
pressure is to use auxiliary reactions containing $O_{2}$ that are easy
to control and are independent of the equilibrium system of interest
except the sharing of the oxygen partial pressure. Two common auxiliary
reactions are the $H_{2}$/$H_{2}O$ and $CO$/$CO_{2}$ mixtures. For the
$H_{2}$/$H_{2}O$ mixture, the chemical reaction is

*Eq. 7‑11* $2H_{2}(gas) + O_{2}(gas) = 2H_{2}O(gas)$

The equilibrium oxygen partial pressure is obtained as

*Eq. 7‑12*
$- RTln\left\{ {\frac{1}{P_{O_{2}}}\left( \frac{P_{H_{2}O}}{P_{H_{2}}} \right)}^{2} \right\} = \mathrm{\Delta}_{\ }^{0}G = \mathrm{\Delta}_{\ }^{0}H - T\mathrm{\Delta}_{\ }^{0}S = - 498488\  + 112.972T\ (J)$

where the $\mathrm{\Delta}_{\ }^{0}H$ and $\mathrm{\Delta}_{\ }^{0}S$
are taken from the substance thermodynamic database (SSUB4) compiled by
Scientific Group Thermodata Europe (SGTE) \[59\], which are slightly
dependent on temperature, and the values in are evaluated at 1273K using
Thermo-Calc \[60\]. At any given temperature, one has the following
relation

*Eq. 7‑13*
$RTlnP_{O_{2}} = \mathrm{\Delta}_{\ }^{0}H - T\mathrm{\Delta}_{\ }^{0}S - 2RTln\frac{P_{H_{2}}}{P_{H_{2}O}} = - 498488 + \left( 112.972 - 2Rln\frac{P_{H_{2}}}{P_{H_{2}O}} \right)T$

Its intercept at $T = 0K$ is given by
$\mathrm{\Delta}_{\ }^{0}H = - 498,488\ (J)$, and the slope by
$- \mathrm{\Delta}_{\ }^{0}S - 2Rln\left( \frac{P_{H_{2}}}{P_{H_{2}O}} \right) = 112.972 - 2Rln\left( \frac{P_{H_{2}}}{P_{H_{2}O}} \right)$.
The values of the $\frac{P_{H_{2}}}{P_{H_{2}O}}$ ratio are marked on
another secondary axis on the right of the Ellingham diagram. The
intersection of the iso-partial-pressure-ratio line and the equilibrium
line of the chemical reaction gives the relation of equilibrium
temperature and equilibrium partial pressure ratio
$\frac{P_{H_{2}}}{P_{H_{2}O}}$ for desired $P_{O_{2}}$ of the chemical
equilibrium.

For the $CO$/$CO_{2}$mixture, the chemical reaction is shown below

*Eq. 7‑14* $2CO(gas) + O_{2}(gas) = 2CO_{2}(gas)$

Similar, from the SSUB database, the following equation is obtained

*Eq. 7‑15*
$- RTln\left\{ {\frac{1}{P_{O_{2}}}\left( \frac{P_{{CO}_{2}}}{P_{CO}} \right)}^{2} \right\} = \mathrm{\Delta}_{\ }^{0}G = \mathrm{\Delta}_{\ }^{0}H - T\mathrm{\Delta}_{\ }^{0}S = - 562,927 - 172.020T\ (J)$

with $\mathrm{\Delta}_{\ }^{0}H$ and $\mathrm{\Delta}_{\ }^{0}S$
calculated at 1273K, which are also slightly temperature-dependent as in
the case of the $H_{2}$/$H_{2}O$ mixture. The iso-partial-pressure-ratio
lines are written as

*Eq. 7‑16*
$RTlnP_{O_{2}} = \mathrm{\Delta}_{\ }^{0}H - T\mathrm{\Delta}_{\ }^{0}S - 2RTln\frac{P_{CO}}{P_{{CO}_{2}}} = - 562,927 + \left( 172.020 - 2Rln\frac{P_{CO}}{P_{{CO}_{2}}} \right)T\ $

Its intercept at $T = 0K$ is given by
$\mathrm{\Delta}_{\ }^{0}H = - 562,927\ (J)$, and the slope by
$172.020 - - 2Rln\left( \frac{P_{CO}}{P_{{CO}_{2}}} \right)$. The values
of the $\frac{P_{CO}}{P_{{CO}_{2}}}$ ratio are marked on the third
secondary axis on the right of the Ellingham diagram. The intersection
of the iso-partial-pressure-ratio line and the equilibrium line of the
chemical reaction gives the relation of equilibrium temperature and
equilibrium partial pressure ratio $\frac{P_{CO}}{P_{{CO}_{2}}}$ for
desired $P_{O_{2}}$ of the chemical equilibrium.

Therefore, one can use a mixture of the $H_{2}$/$H_{2}O$ or
$CO$/$CO_{2}$ to obtain the desired low $P_{O_{2}}$ values using and or
calculating from the SSUB database. For example,
$P_{O_{2}} = 10^{- 15}\ atm$ at 1273K, the calculated values from the
SSUB database are $\frac{P_{H_{2}}}{P_{H_{2}O}} \approx 1.67$ and
$\frac{P_{CO}}{P_{{CO}_{2}}} \approx 2.78$, respectively, in which the
temperature dependences of $\mathrm{\Delta}_{\ }^{0}H$ and
$\mathrm{\Delta}_{\ }^{0}S$ and the many more gaseous species in the gas
phase are taken into account. On the other hand, the reading from the
Ellingham diagram gives $\frac{P_{H_{2}}}{P_{H_{2}O}} \approx 2.0$ and
$\frac{P_{CO}}{P_{{CO}_{2}}} \approx 2.4$, and the agreement with the
more accurate calculations above is remarkable keeping in mind the
uncertainties in graphically drawing the lines and reading both values
in the logarithmic scales from the diagram, indicating the robustness of
the Ellingham diagram.


## Internal process and differential and integrated driving forces

The driving force for an internal process can be defined as following
using $U$, $H$, $F$, or $G$ as discussed in Chapter and depending on
what system variables are kept constant

*Eq. 7‑1*
$- D = \left( \frac{\partial U}{\partial\xi} \right)_{S,\ V,N_{i}\ } = \left( \frac{\partial H}{\partial\xi} \right)_{S,\ P,N_{i}\ } = \left( \frac{\partial F}{\partial\xi} \right)_{T,\ V,N_{i}\ } = \left( \frac{\partial G}{\partial\xi} \right)_{T,P,N_{i}\ }$

This can be termed as differential driving force as it relates the
derivative of energy with respect to an internal process so the change
is infinitesimally small and does not affect the properties of the
system significantly. For a system under constant $T$ and $P$, let us
consider an internal process for forming a new phase $\alpha$ with the
composition $x_{i}^{\alpha}$ and Gibbs energy
$G_{m}^{\alpha}\left( x_{i}^{\alpha} \right)$. The differential driving
force for such an internal process can thus be defined as

*Eq. 7‑2*
$- D = G_{m}^{\alpha}\left( x_{i}^{\alpha} \right) - \sum_{i}^{}x_{i}^{\alpha}\mu_{i} = \sum_{i}^{}x_{i}^{\alpha}\left( \mu_{i}^{\alpha} - \mu_{i} \right)$

where $\mu_{i}$ is the chemical potential of component *i* in the
system. This may also be called as nucleation driving force as if the
$\alpha$ phase is nucleating in the system. As discussed in Chapter ,
chemical potentials are the intercepts on the Gibbs energy axis by the
tangent plane of Gibbs energy. thus represents the distance between the
tangent planes of the original system and the $\alpha$ phase at the
composition of the $\alpha$ phase. Evidently, this distance is at its
maximum when the two tangent planes are parallel to each other, i.e. the
commonly called parallel tangent construction when evaluating nucleation
driving force.

The situation is different for chemical reactions where the amount of
each component in reactants is the same as that in products, i.e. there
is a mass balance between reactants and products. The driving force for
a chemical reaction is defined by the Gibbs energy difference between
the reactants and products as if all the reactants are transferred to
the products. This driving force may thus be called integrated driving
force as it describes the energy difference of a system under two
different states, i.e.

*Eq. 7‑3*
$- \int_{}^{}{Dd\xi} = \mathrm{\Delta}G = \sum_{p}^{}{n_{p}G_{p}} - \sum_{r}^{}{n_{r}G_{r}}$

where superscripts $p$ and $r$ denote products and reactants, $n_{p}$
and $n_{r}$ their corresponding moles, $G_{p}$and $G_{r}$ the Gibbs
energies in per mole of formula of their respective stoichiometries as
written in the chemical reaction. Conventionally, the products and
reactants are represented by species or stoichiometric compounds rather
than individual phases. This is particularly evident when various
gaseous species are considered in a chemical reaction. Consequently,
$G_{p}$and $G_{r}$ in represent the chemical potentials of product and
reactant species. For species with a fixed composition, its chemical
potential is the same as its Gibbs energy as shown by . For a species in
a solution phase, its chemical potential is related to its activity as
shown in . can thus be further written as

*Eq. 7‑4*
$\mathrm{\Delta}G = \sum_{p}^{}{n_{p}_{\ }^{0}G_{p}} - \sum_{r}^{}{n_{r}_{\ }^{0}G_{r}} + RTln\frac{\prod_{p}^{}\left( a_{p} \right)^{n_{p}}}{\prod_{r}^{}\left( a_{r} \right)^{n_{r}}} = \mathrm{\Delta}_{\ }^{0}G + RTln\frac{\prod_{p}^{}\left( a_{p} \right)^{n_{p}}}{\prod_{r}^{}\left( a_{r} \right)^{n_{r}}}$

It is evident from that for stoichiometric phases in a chemical
reaction, their activities equal to one. At equilibrium, the integrated
driving force becomes zero, i.e. $\mathrm{\Delta}G = 0$, and is
re-arranged to become

*Eq. 7‑5*
$- RTln\frac{\prod_{p}^{}\left( a_{p} \right)^{n_{p}}}{\prod_{r}^{}\left( a_{r} \right)^{n_{r}}} = - RTlnK_{e} = \mathrm{\Delta}_{\ }^{0}G = \mathrm{\Delta}_{\ }^{0}H - T\mathrm{\Delta}_{\ }^{0}S$

where $K_{e}$ is often called reaction constant relating the activities
of products and reactants at equilibrium. In a system with
$\frac{\prod_{p}^{}\left( a_{p} \right)^{n_{p}}}{\prod_{r}^{}\left( a_{r} \right)^{n_{r}}} > K_{e}$,
the chemical reaction goes to left, and the products are decomposed,
while
$\frac{\prod_{p}^{}\left( a_{p} \right)^{n_{p}}}{\prod_{r}^{}\left( a_{r} \right)^{n_{r}}} < K_{e}$,
the chemical reaction goes to right, and the products are formed. is
often recast into the following form by dividing $–RT$ on both sides of
the equation

*Eq. 7‑6*
$\ln K_{e} = - \frac{\mathrm{\Delta}_{\ }^{0}H}{RT} + \frac{\mathrm{\Delta}_{\ }^{0}S}{R}$

With $\ln K_{e}$ plotted with respect to $1/T$, indicates that the slope
is $- \mathrm{\Delta}_{\ }^{0}H$/R and the intercept on the y axis is
$\mathrm{\Delta}_{\ }^{0}{S/R}$ as shown in .

Figure ‑: $\ln K_{e}$ plotted with respect to $1/T$ for several M-O
systems at 1 bar and $K_{e}$ represented by the partial pressure ratio
of CO<sub>2</sub> and CO.


## Maximum reaction rate and chemical transport reactions 

Equilibrium thermodynamics can be used to calculate the maximum rates of
reaction that are possible in dynamically reacting systems, such as when
a corrosive gas passes over a heated sample. Other examples of such
reactions include the reduction of a metal oxide in flowing hydrogen,
the evaporation of a material in a vacuum, and the deposition of a thin
film by a chemical vapour deposition process. The basic assumption used
in calculating these maximum reaction rates is that local equilibrium
exists at the location of the considered reaction.

This section examines several examples in which maximum reaction rates
are calculated. The system can typically be divided into three regions:
(1) the input region, which is usually near room temperature, (2) the
high temperature region in which the primary reaction of interest is
occurring, and (3) the exit region. Such a system is almost always at
constant pressure throughout the system. Since the three regions have
different temperatures, the key is to use the mass conservation of the
carrier gas in all three regions. With the input gas at $T = 298K$ and
$P = 1atm$, the volume of one mole of ideal gas is
$0.0244\left( m^{3} \right) = 24.4(liter)$. The number of moles of input
gas flowing through the system can be written as

*Eq. 7‑17*
$n_{gas}^{0} = \frac{Pf_{gas}}{RT} = \frac{f_{gas}}{24.4} = 0.0409f_{gas}$

where $f_{gas}$ is the input gas flow rate in liter per unit time at
$T = 298K$ and $P = 1atm$, and $R$ the gas constant.

The first example is the evaporation of water in a flowing stream of dry
hydrogen. A schematic diagram of the system being considered is given in
. The goal of the calculation is to determine the maximum rate of
$H_{2}O(liquid)$ loss, $n_{H_{2}O}^{\ }$, through vaporization in a
flowing stream of $H_{2}$, e.g. for generating a given $H_{2}O/H_{2}$
ratio for producing a given $O_{2}$ pressure in a high temperature
system as related to the Ellingham diagram discussed in Chapter . The
maximum rate is determined by saturating the $H_{2}$ with the
equilibrium vapour pressure of $H_{2}O(liquid)$ with the number of moles
of $H_{2}$ being $n_{H_{2}}^{0} = 0.0409f_{H_{2}}$ and $f_{H_{2}}$ being
the input flow rate of $H_{2}$ in liter per unit time.

Figure ‑: Schematic diagram of vaporization of water in a flowing stream
of dry hydrogen

In the first input region, the total number moles of gas is
$N = n_{H_{2}}^{0}$. In the second high temperature region with the
temperature and pressure being $T_{sys}$ and $P_{sys}$, the total number
moles of gas is $N = n_{H_{2}}^{0} + n_{H_{2}O}^{\ }$, which is the same
for the third exit region with $T_{exit}$. To avoid condensation, one
needs to maintain $T_{exit} > T_{sys}$. The vapour pressure of $H_{2}O$
at $T_{sys}$, $P_{H_{2}O\ }$, can be obtained from equilibrium
thermodynamic calculations. Since the $H_{2}$ and $H_{2}O$ are occupying
the same volume at the same temperature, the maximum number of moles
$H_{2}O$ can be calculated from the following relation based on the
ideal gas law

*Eq. 7‑18*
$n_{H_{2}O}^{\ } = \frac{P_{H_{2}O}^{\ }}{P_{H_{2}}^{\ }}n_{H_{2}}^{0} = \frac{P_{H_{2}O}^{\ }}{P_{sys}^{\ } - P_{H_{2}O}^{\ }}n_{H_{2}}^{0} = \frac{P_{H_{2}O}^{\ }}{P_{sys}^{\ } - P_{H_{2}O}^{\ }}\frac{f_{H_{2}}}{24.4}$

For $P_{sys}^{\ } = 101325Pa$, one can calculate $P_{H_{2}O}^{\ }$ and
plot $\frac{n_{H_{2}O}^{\ }}{f_{H_{2}}}$ as a function of temperature
from the SSUB database as shown in . The corresponding partial pressure
ratio, $P_{H_{2}O}/P_{H_{2}}$ , is plotted in , which can be used to
calculate $P_{O_{2}}^{\ }$ at any given temperatures. An example is
shown in with $T_{sys} = 348.15K$ and $P_{H_{2}O}/P_{H_{2}} = 0.607$.

Figure ‑: Ratio of maximum number of moles $H_{2}O$ with respect to
hydrogen flow rate, $\frac{n_{H_{2}O}^{\ }}{f_{H_{2}}}$, plotted as a
function of temperature.

Figure ‑: Partial pressure ratio, $P_{H_{2}O}/P_{H_{2}}$, corresponding
to .

Figure ‑: Partial pressure of oxygen, $P_{O_{2}}^{\ }$, as a function of
temperature with $T_{sys} = 348.15K$ and $P_{H_{2}O}/P_{H_{2}} = 0.607$.

The second example is the corrosion of $SiO_{2}$(s) by flowing $H_{2}$
gas at high temperatures. Considering $T_{sys}^{\ } = 1700K$,
$P_{sys}^{\ } = 101325Pa$, and $f_{H_{2}} = 1\ liter/min$, the system is
thus defined with 0.0409 moles of $H_{2}$ and an equilibrium between the
gas phase and the tridymite $SiO_{2}$. The equilibrium calculation gives
0.04092 moles of gas with its constitutions as
$H_{2}:H_{2}O:SiO = 0.998887:4.833 \bullet 10^{- 4}:4.832 \bullet 10^{- 4}$.
The corrosion rate of SiO<sub>2</sub>(s) is thus
$0.04092x4.832 \bullet 10^{- 4} = 1.98 \bullet 10^{- 5}mol/min = 1.19gram/min.$

Another example is to consider that the flowing $CO$ gas of 298K and
1atm ($= 101325Pa$) at a rate of $1\ liter/min$ passes through and
equilibrates with single phase $C(s)$ at 1500K. The equilibrium system
is defined by $T = 1500K$, $P = 1atm$, and 0.0409 moles of $CO$ with the
equilibrium between the gas phase and C(s). The equilibrium calculation
gives 0.040872 moles of gas phase with the mole fraction of $CO$ being
0.999327, resulting in the loss of *CO* or the deposition of *C(s)* at a
rate of
$0.0409 - 0.040872 \bullet 0.999327 = 5.55 \bullet 10^{- 5}mole/min$.

A chemical transport reaction is a reaction in which a condensed phase
reacts with a gas phase to form vapour-phase products, which in turn
undergo the reverse reformation of the condensed phase. Two well-known
examples of such reactions are

*Eq. 7‑19* $M(s) + \frac{n}{2}I_{2}(g) = MI_{n}(g)$

*Eq. 7‑20* $Ni(s) + 4CO(g) = Ni{(CO)}_{4}(g)$

which are used in the purification of metals by the iodide process and
in the purification of nickel by the Mond-Langer process. In both
processes the forward reaction is favoured by lower temperatures and the
reverse reaction by higher temperatures, resulting in the deposition of
the metal. The most common technique for causing a chemical transport of
a condensed substance makes use of the temperature dependence of the
equilibrium constant. As was discussed previously, the enthalpy of
reaction, $\mathrm{\Delta}_{\ }^{0}H$, determines the manner in which
$K_{e}$ changes with temperature (see ). The value of $K_{e}$ increases
with increasing *T* for $\mathrm{\Delta}_{\ }^{0}H > 0$, $K_{e}$
decreases with increasing *T* for $\mathrm{\Delta}_{\ }^{0}H < 0$, and
$K_{e}$ is independent of T for $\mathrm{\Delta}_{\ }^{0}H = 0$. The
$\mathrm{\Delta}_{\ }^{0}H$ and $\mathrm{\Delta}_{\ }^{0}S$ values for
chemical transport reactions may be either positive or negative. For
reactions by both are positive, and for reactions by both are negative.

In a typical experiment the starting solid is located at the point in a
temperature gradient that corresponds to the largest $K_{e}$ value for
the experimental condition. As the gaseous species migrate to other
locations in the system with temperatures corresponding to lower $K_{e}$
values, the reverse reaction occurs to satisfy the new equilibrium
requirements, and the solid phase is deposited. The dependence of
$K_{e}$ on $\mathrm{\Delta}_{\ }^{0}H$ results in a material transport
from hot to cold for $\mathrm{\Delta}_{\ }^{0}H > 0$ (the same as for
vaporization-condensation reactions), from cold to hot for
$\mathrm{\Delta}_{\ }^{0}H < 0$, and no transport for
$\mathrm{\Delta}_{\ }^{0}H = 0$.

The success of a particular reaction in causing an appreciable transport
of a condensed phase depends mainly upon the partial pressure gradients
or concentration gradients of the gaseous species in the system. A
reaction whose equilibrium is extreme toward either the reactant side or
the product side will not give an appreciable transport of material. The
concentration gradients are too small in such a system. Reactions with
equilibrium constants near unity at the experimental temperatures
usually give the largest transport since small changes in $K_{e}$ cause
large changes in concentrations. The general condition required for
obtaining a $K_{e}$ value near unity at a reasonable temperature is that
$\mathrm{\Delta}_{\ }^{0}H$ and $\mathrm{\Delta}_{\ }^{0}S$ both have
the same sign, resulting from the equalities of .


## Trends of entropies of reactions

The reaction entropy, $\mathrm{\Delta}_{\ }^{0}S$ in , plays an
important role in determining equilibria of high-temperature reactions.
The most important single factor that determines the entropy of a
reaction is the net change in the number of moles of gas as briefly
mentioned in the discussion of the Ellingham diagram above. The reason
this is true can be explained as follows.

The entropy of a substance can be thought of as being the sum of four
parts: (i) translational, (ii) rotational, (iii) vibrational, and (iv)
electronic. The translational entropy of a gas is the largest entropy
term under most conditions. To the extent that the other contributions
cancel between reactants and products, the entropy of reaction is
determined by the change in the number of moles of gaseous molecules.
Based on the literature data or calculations from the SSUB database, the
net change in the number of moles of gas in a reaction results
approximately in an entropy of reaction of about 175±45 J/K/mole-gas at
298K for many halides and oxides. The chemical reactions of and
discussed above both reduce the gas by one mole, and their entropies of
reaction are -113 and -172 J/K at 1273K, and -89 and -173J/K at 298K,
respectively, indicating that the chemical reaction of is an exception
of the empirical rule. For chemical reactions shown in the Ellingham
diagram, their entropies of reaction follow this empirical rule pretty
well with some of them shown in calculated from the SGTE database.

Table ‑: Entropies of reactions with gas at 298.15K, J/K

Reaction: Si+O<sub>2</sub> =SiO<sub>2</sub> -182

Reaction: Ti+O<sub>2</sub>=TiO<sub>2</sub> -185

Reaction: 2Mg+O<sub>2</sub>=2MgO -217

Reaction: 2Ca+O<sub>2</sub>=2CaO -212

Reaction: 2Mn+O<sub>2</sub>=2MnO -150

Since the entropy of a reaction is primarily determined by the net
change in the number of moles of gas, the entropies for reactions
involving only condensed phases must be small. The entropies of fusion
of monatomic solids are usually in the range 8-15 J/K/mole-atom as shown
for some elements in . Most metals and many ionic salts have values that
lie in this range when given in terms of per mole of atom of material.
There are few exceptions such as silicon and boron shown in the table.
For solid-state reactions, the average values can be approximated as
0±8J/K/mole-atom as also shown in the table.

Table ‑: Entropies of reactions of condensed phases at 298.15K, J/K

Reaction: Si(s)=Si(l) 29.762

Reaction: Ti(s2)=Ti(l) 7.288

Reaction: Mg(s2)=Mg(l) 9.184

Reaction: Ca(s2)=Ca(l) 7.659

Reaction: Mn(s2)=Mn(l) 11.443

Reaction: W(s)=W(l) 14.158

Reaction: B(s)=B(l) 21.380

Reaction: 3Fe+C=CFe<sub>3</sub> 17.060

Reaction: S+Mn=MnS 13.909

Reaction: NiO+Fe<sub>2</sub>O<sub>3</sub>=Fe<sub>2</sub>NiO<sub>4</sub>
0.464


## Aqueous solution and Pourbaix diagram

The importance of aqueous solutions in all aspects of life is so well
known and needs not be discussed further. Since many electrochemical
processes involve electrolyte solutions in an aqueous solvent,
electrochemical processes including water, hydrogen, and/or oxygen are
discussed in more details. The hydrogen-oxygen cell can be described for
both acidic electrolytes and alkaline electrolytes. With acidic
electrolytes, H<sup>+</sup> is in much higher concentrations than
OH<sup>-</sup>, and thus half-cell reactions with H<sup>+</sup> as an
ionic transport species are more important than those involving
OH<sup>-</sup>. The reverse is true for alkaline electrolytes that
contain high OH<sup>-</sup>concentrations. Other than for nearly neutral
acid-base systems, either H<sup>+</sup> or OH<sup>-</sup> dominates the
other by several orders of magnitude as can be seen from the value of
the 298 K dissociation constant for H<sub>2</sub>O:

*Eq. 8‑35* H<sub>2</sub>O(l) = H<sup>+</sup>(aq) + OH<sup>-</sup>(aq)

with the reaction constant being K<sub>e</sub> =
\[H<sup>+</sup>\]\[OH<sup>-</sup>\] = 10<sup>-14</sup> and
$\mathrm{\Delta}_{\ }^{0}G$= -RT *ln* K<sub>e</sub> *β*= +79,908 J. By
convention, one defines pH = - log \[H<sup>+</sup>\] and pOH = - log
\[OH<sup>-</sup>\], and then pH + pOH = 14.

Under acidic electrolyte conditions of low pH (high \[H<sup>+</sup>\]
concentrations) the anode reaction in a hydrogen-oxygen cell is:

*Eq. 8‑36* ½ H<sub>2</sub>(g) = H<sup>+</sup>(aq) + e<sup>-</sup>

with ε<sub>1</sub><sup>o</sup> = 0.0 V and
$\mathrm{\Delta}_{\ }^{0}G_{1}$= 0 J. The corresponding cathode
(reduction) reaction is:

*Eq. 8‑37* 2 H<sup>+</sup>(aq) + ½ O<sub>2</sub>(g) + 2 e<sup>-</sup> =
H<sub>2</sub>O(l)

with ε<sub>2</sub><sup>o</sup> = 1.229 V and
$\mathrm{\Delta}_{\ }^{0}G_{2}$ = -2\*1.229\*96,485 J = -237,160 J. The
net cell reaction for acidic electrolytes is:

*Eq. 8‑38* H<sub>2</sub>(g) + ½ O<sub>2</sub>(g) = H<sub>2</sub>O(l)

with ε<sup>o</sup><sub>cell</sub> = 1.229 V and
$\mathrm{\Delta}_{\ }^{0}G_{cell}$ = -2\*1.229\*96,485 J = -237,160 J

Under alkaline electrolyte conditions of high pH (high
\[OH<sup>-</sup>\] concentrations) the anode reaction in a
hydrogen-oxygen cell is:

*Eq. 8‑39* 2 OH<sup>-</sup>(aq) + H<sub>2</sub>(g) = 2
H<sub>2</sub>O(l) + 2 e<sup>-</sup>

with ε<sub>3</sub><sup>o</sup> = 0.828 V and
$\mathrm{\Delta}_{\ }^{0}G_{3}$= -2\*0.828\*96,485 J = -159,779 J. The
corresponding cathode (reduction) reaction is:

*Eq. 8‑40* H<sub>2</sub>O(l) + ½ O<sub>2</sub>(g) + 2 e<sup>-</sup> = 2
OH<sup>-</sup>(aq)

with ε<sub>4</sub><sup>o</sup> = 0.401 V and
$\mathrm{\Delta}_{\ }^{0}G_{4}$ = -2\*0.401\*96,485 J = -77,381 J. The
net cell reaction for alkaline electrolytes is:

*Eq. 8‑41* H<sub>2</sub>(g) + ½ O<sub>2</sub>(g) = H<sub>2</sub>O(l)

with ε<sup>o</sup><sub>cell</sub> = 1.229 V and
$\mathrm{\Delta}_{\ }^{0}G_{cell}$ = -2\*1.229\*96,485 J = -237,160 J.

Plots of ε versus pH for a given chemical system have been typically
used to exhibit the stability relationships of ionic species and solid
phases in aqueous-based electrochemical systems. These graphs are often
called Pourbaix diagrams after the inventor and are at constant
temperature and constant pressure diagrams for a constant concentration,
usually for one metallic element. By convention, the ε in a Pourbaix
diagram corresponds to the potential for the cathode reduction reactions
in the electrochemical half-cell with electrons as reactants. Pourbaix
diagrams can be extended to multi-component materials when thermodynamic
properties of the components are available in both the materials and the
aqueous solution.

An example of an ε versus pH diagram is shown in for the
Ni-H<sub>2</sub>O system at a 298K, 1 bar, and $c_{{Ni}^{2 +}} = 0.001$
molality. Three stability regions for Ni species are shown: Ni(s),
NiO(s), and \[Ni<sup>2+</sup>\]. The two dashed lines on this diagram
correspond to hydrogen reduction () and oxygen reduction () reactions,
respectively.

Figure ‑: An ε versus pH, Pourbaix diagram for the Ni-H<sub>2</sub>O at
298K, 1 bar, and $c_{{Ni}^{2 +}} = 0.001$ molality.

For the ε and pH conditions within the boundaries of the Ni(s) region,
no solid phase other than Ni(s) is stable, no ionic species with a
concentration of 1 molarity is stable, and no gas species with a
pressure of 1 bar is stable. Similar statements could be made about the
NiO(s) and \[Ni<sup>2+</sup>\] areas on the diagram. In the
\[Ni<sup>2+</sup>\] area, introduction of Ni(s) or NiO(s) into the
system would result in the dissolution of these solid phases since they
are not stable with respect to the \[Ni<sup>2+</sup>\] aqueous solution.
The corresponding chemical reactions proceed spontaneously to the right
as follows until the solid phases are consumed:

*Eq. 8‑42* Ni(s) → Ni<sup>2+</sup>(1 molarity) + 2 e<sup>-</sup>

*Eq. 8‑43* NiO(s) + 2 H<sup>+</sup>(aq) → Ni<sup>2+</sup>(1 molarity) +
H<sub>2</sub>O(l)

No H<sup>+</sup>(aq) in involved in the first reaction, , so the
boundary line separating Ni(s) and Ni<sup>2+</sup> is independent of pH.
No oxidation or reduction occurs in the second reaction, , i.e. no
electrons are reactants or products in the reaction, the boundary line
separating NiO(s) and Ni<sup>2+</sup> is independent of ε.

Note the convention that the ε is the potential for a cathode reduction
reaction, and boundary lines between two stability regions depict
conditions under which partial equilibrium occurs for the two species
for the ε and pH values at any point on these lines. For the boundary
line separating Ni(s) and Ni<sup>2+</sup> in an ideal aqueous solution,
i.e. the reverse of , the following equation is obtained.

*Eq. 8‑44* ε = ε<sup>o</sup> = -0.268 V

For the NiO(s)-Ni<sup>2+</sup> boundary line of an ideal solution, the
reaction, , is a complete equilibrium, and thus the relationship is

*Eq. 8‑45*
$0 = \mathrm{\Delta}_{\ }^{0}G + RTln\frac{1}{\left( c_{H^{+}} \right)^{2}} = \mathrm{\Delta}_{\ }^{0}G + 2 \cdot 2.303 \cdot RT \cdot pH$

*Eq. 8‑46*
$pH = - \frac{\mathrm{\Delta}_{\ }^{0}G}{2 \cdot 2.303 \cdot RT}$

where $\mathrm{\Delta}_{\ }^{0}G$ is obtained as follows and can be
calculated from the SSUB database and the standard potential of Ni,

*Eq. 8‑47*

At a specified temperature, only one standard free energy and only one
equilibrium constant exists for this chemical reaction, and thus only
one specific value of $pH = 6.631$ exists for the reaction represented
by in this Pourbaix diagram.

The diagonal line in represents the equilibrium between Ni(s) and NiO(s)
and is for a partial equilibrium reaction that is the sum of reactions
of and

*Eq. 8‑48* NiO(s) + 2 H<sup>+</sup>(aq) + 2 e<sup>-</sup> ═ Ni(s) +
H<sub>2</sub>O(l)

The reduction of Ni from a divalent state in NiO to metallic Ni(s)
occurs, but the reaction also depends on the H<sup>+</sup>
concentration, the pH. The corresponding Gibbs energy and Nernst
equations are,

*Eq. 8‑49*
$\mathrm{\Delta}G = \mathrm{\Delta}_{\ }^{0}G + RTln\frac{1}{\left( c_{H^{+}} \right)^{2}} = - 23,939 + 2 \cdot 2.303 \cdot RT \cdot pH$

*Eq. 8‑50*
$\varepsilon\  = \ \varepsilon^{0}\  - \ \frac{RT}{2f}\ln\frac{1}{\left( c_{H^{+}} \right)^{2}}\  = 0.124\ –\ \frac{2.303 \cdot RT}{f}pH$

where $\mathrm{\Delta}_{\ }^{0}G$ can be calculated as follows

*Eq. 8‑51*

The two additional lines in correspond to the reduction reactions
related to H<sub>2</sub> and O<sub>2</sub> gases, i.e. the stability of
H<sub>2</sub>O. The lower one is for the reverse of under ε<sup>o</sup>
= 0 and $P_{H_{2}} = 1$ with the Nernst equation being

*Eq. 8‑52*
$\varepsilon\  = \ \varepsilon^{0}\  - \ \frac{RT}{f}\ln\frac{\left( P_{H_{2}} \right)^{1/2}}{c_{H^{+}}}\  = - \frac{2.303 \cdot RT}{f}pH$

As the pH increases from 0, ε becomes more negative as is depicted. The
top dashed line corresponds to the oxygen reduction reaction represented
by under ε<sup>o</sup> = 1.225 calculated from the aqueous solution
database in Thermo-Calc \[60\] and $P_{O_{2}} = 1$ with the Nernst
equation being

Eq. 8‑53
$\varepsilon\  = \ \varepsilon^{0}\  - \ \frac{RT}{2f}\ln\frac{\left( P_{O_{2}} \right)^{1/2}}{\left( c_{H^{+}} \right)^{2}}\  = 1.225\ –\ \frac{2.303 \cdot RT}{f}pH$

The dependence of ε on pH is identical for both reduction reaction and ,
and their intercepts at $pH = 0$ differ by their difference in their
ε<sup>o</sup> values.

In this simple Pourbaix diagram of Ni in an ideal aqueous solution, all
boundary lines are straight because there is only one ionic species of
Ni in the aqueous solution, i.e. Ni<sup>2+</sup>. When there are more
than one ionic species in the aqueous solution, the boundary lines may
no longer be straight due to the competition between species. One
example is Cu with two main ionic species of Cu<sup>+2</sup> and
CuOH<sup>+</sup>, and the reduction reaction between the metallic Cu and
the aqueous solution involves both two species, i.e.

*Eq. 8‑54*
${xCu}^{2 + \ }\  + \ (1 - x){CuOH}^{+} + (1 - x)H^{+} + 2\ e - \ \  = \ \ Cu(s) + {(1 - x)H}_{2}O$

with

*Eq. 8‑55*
$\mathrm{\Delta}G = \mathrm{\Delta}_{\ }^{0}G + RTln\frac{1}{\left( c_{{Cu}^{2 +}} \right)^{x}\left( c_{{CuOH}^{+}}c_{H^{+}} \right)^{1 - x}} = \mathrm{\Delta}_{\ }^{0}G + RTln\frac{1}{\left( c_{{Cu}^{2 +}} \right)^{x}\left( c_{{CuOH}^{+}} \right)^{1 - x}} + 2.303(1 - x) \cdot RT \cdot pH$

*Eq. 8‑56*
$\varepsilon = \varepsilon^{0} - \frac{RT}{2f}\ln\frac{1}{\left( c_{{Cu}^{2 +}} \right)^{x}\left( c_{{CuOH}^{+}} \right)^{1 - x}} - \frac{2.303(1 - x) \cdot RT}{2f}pH$.

It is evident that both the slope and the intercept at $pH = 0$ are a
function of the concentration of ${CuOH}^{+}$, which is a function of
$pH$. Consequently, the boundary between the metallic Cu and the aqueous
solution is no longer a straight line as shown in .

Figure ‑: An ε versus pH, Pourbaix diagram for the Cu-H<sub>2</sub>O
system at 298K, 1 bar, and $c_{Cu} = 0.001$ molality.

The concentrations of various species in the aqueous solution, i.e.
commonly called speciation, are plotted in , showing the change of
dominant species as a function of pH value.

Figure ‑: Concentrations of ionic species in the aqueous solution at
$\varepsilon = 0.3\ V$ from .

In Pourbaix diagrams for alloys with two or more elements, activities of
individual elements are to be used in calculating the potentials of
reduction reactions. Considering a Fe-Ni alloy with Fe<sup>2+</sup> and
Ni<sup>2+</sup> in the aqueous solution, the reduction reactions for Fe
and Ni can be written separately as

*Eq. 8‑57* Ni<sup>2+</sup>(c<sub>Ni</sub>) + 2 e<sup>-</sup> → Ni
(a<sub>Ni</sub> in alloy)

*Eq. 8‑58* Fe<sup>2+</sup>(c<sub>Fe</sub>) + 2 e<sup>-</sup> → Fe
(a<sub>Fe</sub> in alloy)

with their potentials as

*Eq. 8‑59*
$\varepsilon_{Ni}\  = \ \varepsilon_{Ni}^{0} - \frac{2.303RT}{2f}\ln\frac{a_{Ni}}{c_{Ni}} = \  - 0.268 - \frac{2.303RT}{2f}\ln\frac{a_{Ni}}{c_{Ni}}$

*Eq. 8‑60*
$\varepsilon_{Fe}\  = \ \varepsilon_{Fe}^{0} - \frac{2.303RT}{2f}\ln\frac{a_{Fe}}{c_{Fe}} = \  - 0.441 - \frac{2.303RT}{2f}\ln\frac{a_{Fe}}{c_{Fe}}$

In principle, there are two scenarios for a given set of $a_{Ni}$ and
$a_{Fe}$ of the alloy. The first scenario is at the limit of a dilute
aqueous solution, i.e. $c_{Ni} = c_{Fe} = 0.001\ $molarity,
$\varepsilon_{Ni}$ and $\varepsilon_{Fe}$ can be calculated, and the
element with the lower potential has the tendency to dissolve first,
which can result in the so-called dialloying effect. The second scenario
is for the equal potential, i.e. $\varepsilon_{Ni} = \varepsilon_{Fe}$
due to the externally imposed potential, and the equilibrium
concentrations of Fe<sup>+2</sup> and Ni<sup>+2</sup> can be calculated
from and .


...

### Electrical batteries

Batteries utilize electrochemical reactions to generate electricity for
various devices. The theoretic voltage of a battery can be calculated
from and , i.e.

*Eq. 8‑73*
$\varepsilon = - \frac{\mathrm{\Delta}G}{zf} = \varepsilon^{0} - \frac{RTlnQ}{zf}$

with $\mathrm{\Delta}G$ being the driving force of the net cell reaction
and $Q$ being the reaction activity quotient. The actual voltage of a
battery is lower than the theoretical one due to kinetic limitations of
cell reactions and resistance to ion diffusion through the electrolyte.
Based on whether the cell reactions are reversible or not, batteries
typically categorized as either primary disposable or secondary
rechargeable batteries. The net cell reactions in primary disposable
batteries are not easily reversible, and electrode materials may not
return to their original forms by applying a higher external potential
of opposite sign. Consequently, primary batteries cannot be reliably
recharged. On the other hand, the net cell reactions in secondary
batteries are easily reversible. Furthermore, two half-cells in
batteries may use different electrolytes with each half-cell enclosed in
a container and a separator permeable to conducting ions but not the
bulk of the electrolytes.

One common primary battery is zinc-carbon battery with a zinc anode
cylinder and a carbon cathode central rod. The electrolytes are ammonium
or zinc chloride next to the zinc anode and a mixture of ammonium
chloride and manganese dioxide next to the carbon cathode. The half cell
and net reactions with ammonium chloride are as follows

*Eq. 8‑74* Zn + 2NH<sub>3</sub> →
Zn(NH<sub>3</sub>)<sub>2</sub><sup>2+</sup> + 2 e<sup>-</sup>

*Eq. 8‑75* 2NH<sub>4</sub>Cl + 2MnO<sub>2</sub> + 2 e<sup>-</sup> →
2NH<sub>3</sub> + Mn<sub>2</sub>O<sub>3</sub> +
H<sub>2</sub>O+2Cl<sup>−</sup>

*Eq. 8‑76* Zn + 2MnO<sub>2</sub> + 2NH<sub>4</sub>Cl →
Mn<sub>2</sub>O<sub>3</sub> +
Zn(NH<sub>3</sub>)<sub>2</sub>Cl<sub>2</sub> + H<sub>2</sub>O.

The electric potential of the reaction is, treating all compounds as
stoichiometric compounds

Eq. 8‑77
$\varepsilon = - \frac{\mathrm{\Delta}G}{2f} = - \frac{\mathrm{\Delta}^{0}G}{2f} = \frac{1}{2f}\left(_{\ }^{0}G^{Zn} + 2_{\ }^{0}G^{{MnO}_{2}} + 2_{\ }^{0}G^{{NH}_{4}Cl} -_{\ }^{0}G^{H_{2}O} -_{\ }^{0}G^{Zn\left( {NH}_{3} \right)_{2}{Cl}_{2}} -_{\ }^{0}G^{{{Mn}_{2}O}_{3}} \right)$.

The Gibbs energy of $Zn\left( {NH}_{3} \right)_{2}{Cl}_{2}$ is not
available in current databases and has been recently estimated to be
−505,375 J/mole-formula \[62\]. The value of at 298.15K is thus obtained
as 1.67 V, which is pretty close to the actual operating voltage of the
battery around 1.5 V.

While with zinc chloride, the cell reactions and electric potential may
be written as

Eq. 8‑78 Zn + ZnCl<sub>2</sub> + 2OH<sup>−</sup> → 2ZnOHCl + 2
e<sup>-</sup>

*Eq. 8‑79* MnO<sub>2</sub> + H<sub>2</sub>O + e<sup>-</sup> → MnOOH +
OH<sup>-</sup>

*Eq. 8‑80* Zn + 2 MnO<sub>2</sub> + ZnCl<sub>2</sub> + 2 H<sub>2</sub>O
→ 2 MnOOH + 2 ZnOHCl

*Eq. 8‑81*
$\varepsilon = \frac{1}{2f}\left(_{\ }^{0}G^{Zn} + 2_{\ }^{0}G^{{MnO}_{2}} +_{\ }^{0}G^{Zn{Cl}_{2}} + 2_{\ }^{0}G^{H_{2}O} - 2_{\ }^{0}G^{MnOOH} - 2_{\ }^{0}G^{ZnOHCl} \right)$

Secondary batteries can be recharged by applying an external electrical
potential, which reverses the net cell reaction that occur during
discharging. The oldest form of rechargeable battery is the lead-acid
batteries used in automotive, and the latest development is the
lithium-ion (Li-ion) batteries. A lead-acid battery typically uses Pb
and PbO2 as the cathode and anode electrodes and a 35% sulfuric acid and
65% water solution as the electrolyte. Its anode and cathode reactions
can be simplified as follows

Eq. 8‑82 $Pb + SO_{4}^{2 -} = PbSO_{4} + 2e^{-}$

Eq. 8‑83 $PbO_{2} + 4H^{+} + SO_{4}^{2 -} + 2e^{-} = PbSO_{4} + 2H_{2}O$

The net cell reaction is

Eq. 8‑84 $Pb + PbO_{2} + 2H_{2}SO_{4}^{\ } = 2PbSO_{4} + 2H_{2}O$.

Its electric potential is represented by the following equation

*Eq. 8‑85*
$\varepsilon = - \frac{1}{2f}\left( 2_{\ }^{0}G^{H_{2}O} + 2_{\ }^{0}G^{{PbSO}_{4}} -_{\ }^{0}G^{Pb} -_{\ }^{0}G^{{PbO}_{2}} - 2_{\ }^{0}G^{H_{2}SO_{4}} \right)$

with the value at 298.15K being 2.651 V calculated from Thermo-Calc
\[60\]. During discharging, the reaction goes to right, and $PbSO_{4}$
is formed on both anode and cathode, while during charging, the reaction
goes to the left, and $Pb$ and $PbO_{2}$ are restored. In practical
applications, other ionic species such as ${H_{3}O}^{+}$ and
$HSO_{4}^{-}$ may form in the electrolyte, complicating the reactions
and affecting its potential.

In lithium ion batteries, lithium ions migrate in electrolytes between
electrodes made of intercalated lithium compounds during charging and
discharging. LiCoO<sub>2</sub> and LiFePO<sub>4</sub> are two of the
several cathode materials used in lithium ion batteries, and the anode
is typically made of carbon or metallic Li. The anode and cathode
reactions for LiCoO2 batteries can be written in simple forms as follows

Eq. 8‑86 ${Li}_{x}C_{6} = x{Li}^{+} + xe^{-} + 6C$

Eq. 8‑87 $x{Li}^{+} + xe^{-} + {Li}_{1 - x}CoO_{2} = LiCoO_{2}$

with the net reaction and electric potential being

Eq. 8‑88 ${Li}_{x}C_{6} + {Li}_{1 - x}CoO_{2} = LiCoO_{2} + 6C$

*Eq. 8‑89*
$\varepsilon = - \frac{1}{xf}\left\{ 6_{\ }^{0}G^{C} +_{\ }^{0}G^{{LiCoO}_{2}} - G^{{{Li}_{x}C}_{6}} - G^{{{Li}_{1 - x}CoO}_{2}} \right\} = - \frac{1}{f}\left\{ \left( \mu_{Li}^{{Li}_{1 - x}CoO_{2}} - \mu_{Li}^{{Li}_{x}C} \right) - \frac{1}{x}\left( \mu_{LiCoO_{2}}^{{Li}_{1 - x}CoO_{2}} -_{\ }^{0}G^{{LiCoO}_{2}} \right) \right\}$

The electric potential is a function of $x$. The value in the first
parenthesis in the above equation denotes the chemical potential
difference of Li between two electrodes, and the value in the second
parenthesis represents the chemical potential difference of
${LiCoO}_{2}$ between the states in the solution phase of
${Li}_{1 - x}CoO_{2}$ and by itself. Gibbs energies of ${{Li}_{x}C}_{6}$
and ${{Li}_{1 - x}CoO}_{2}$ need to be obtained as a function $x$ in
order to calculate the electric potential of the battery.

LiFePO<sub>4</sub> uses metallic lithium as the anode with following
half-cell and net cell reactions

Eq. 8‑90 $xLi = x{Li}^{+} + xe^{-}$

Eq. 8‑91 $x{Li}^{+} + xe^{-} + {Li}_{1 - x}FePO_{4} = LiFePO_{4}$

Eq. 8‑92 $xLi + {Li}_{1 - x}FePO_{4} = LiFePO_{4}$.

Its electric potential is also a function of $x$, i.e.

*Eq. 8‑93*
$\varepsilon = - \frac{1}{xf}\left\{_{\ }^{0}G^{LiFePO_{4}} - x\ ^{0}G^{Li} - G^{{Li}_{1 - x}FePO_{4}} \right\} = - \frac{1}{f}\left\{ \left( \mu_{Li}^{{Li}_{1 - x}FePO_{4}} - \ ^{0}\mu_{Li} \right) - \frac{1}{x}\left( \mu_{LiFePO_{4}}^{{Li}_{1 - x}FePO_{4}} -_{\ }^{0}G^{LiFePO_{4}} \right) \right\}$

The value in the first parenthesis in the above equation denotes the
chemical potential difference of Li between two electrodes, and the
value in the second parenthesis represents the chemical potential
difference of $LiFePO_{4}$ between the states in the solution phase of
${Li}_{1 - x}FePO_{4}$ and by itself. Consequently, Gibbs energy of
${Li}_{1 - x}FePO_{4}$ needs to be obtained as a function $x$ in order
to calculate the electric potential of the battery. It is known that
there are several miscibility gaps in the $FePO_{4}$ and $LiFePO_{4}$
psuedo-binary system, in which the chemical potentials are constants, so
is the electric potential, resulting in stable battery output.


## Electrolyte reactions and electrochemical reactions

Electrolytes that dissolve in a polar solvent such as water to produce
ionic species do not necessarily exhibit valence changes. A simple
example is the strong electrolyte NaCl(s) dissolving in water to produce
solvated ions

*Eq. 8‑1* $NaCl(s)\ \  = \ \ {Na}^{+}(aq)\ \  + \ \ {Cl}^{–}(aq)$

where the $(aq)$ indicates the ionic species in an aqueous solution. In
this system, the ion concentrations must become quite large before the
solution is saturated and can exist in equilibrium with $NaCl(s)$. Its
reaction constant, defined by , is shown as
$K_{e} = \ \ a_{{Na}^{+}}a_{{Cl}^{-}}$. If the product of the ion
activities is less than $K_{e}$, the solution is not saturated, and more
$NaCl(s)\ \ $ can be dissolved.

The precipitation of $AgCl(s)$, a weak electrolyte, occurs quite readily
when ${Cl}^{–}$ ions are added to an aqueous solution containing
${Ag}^{+}(aq)$:

*Eq. 8‑2* ${Ag}^{+}(aq)\  + \ \ {Cl}^{–}(aq)\  = AgCl(s)$

The equilibrium constant for this reaction,
$K_{e} = \ \frac{1}{\left( a_{{Ag}^{+}}a_{{Cl}^{-}} \right)}$ is quite
large, so the equilibrium product of the ion activities, proportional to
their concentrations, is quite small. In the laboratory, the above
reaction could occur as a result of adding hydrochloric acid to a silver
nitrate solution. The accompanying $H^{+}(aq)$ \[or ${H_{3}O}^{+}(aq)$\]
and ${NO_{3}}^{-}(aq)$ ions in solution are not directly involved in the
silver chloride precipitation reaction so are not shown in reaction
represented by .

The above ionic equilibria in the $AgCl(s) - H_{2}O$ system is not only
important for understanding this electrolyte system, but also critical
in electrochemical systems in which *Ag(s)* undergoes a valence change
at one electrode and reacts with a ${Cl}^{–}(aq)$ ion to produce
*AgCl(s)*, and an electron that is externally transported finite
distances to another electrode. The *oxidation* reaction occurs at the
Ag/AgCl electrode (*anode* half-cell reaction where electrons are
*added* into the system)

*Eq. 8‑3* $Ag(s)\ \  + \ {Cl}^{-}(aq)\ \  = \ \ AgCl(s)\ \  + \ \ e^{-}$

A *reduction* reaction occurs at the other electrode (*cathode*
half-cell reaction where electrons are *consumed* by the reaction)

*Eq. 8‑4* $\frac{1}{2}{Cl}_{2}(g) + \ \ e^{-} = \ {Cl}^{-}(aq)\ \ $

The *net cell reaction* results in the formation AgCl(s) from its
elements

*Eq. 8‑5* $Ag(s)\ \  + \frac{1}{2}{Cl}_{2}\ (g)\  = \ \ AgCl(s)$

Without knowledge of the physical system under which the reaction is
occurring, it would not be possible to know if reaction of was a result
of chlorine gas reacting directly with Ag(s), or if the reaction was
part of an electrochemical cell with a transport of electrons and ions
over finite distances. The addition of the two half-cell reactions gives
the *net cell reaction*, which does not show electrons as either
reactant or product species and may or may not include ionic species in
the reaction*.* A schematic of an electrochemical cell for the above
system is shown in Figure 8‑1.

Figure ‑: Schematic diagram of an electrochemical cell consisting of a
chlorine electrode and a silver-silver chloride electrode.

Oxidation and reduction can occur in electrolyte reactions without
creating an electrochemical cell. This is the case when chlorine gas
reacts directly with silver on a Ag(s) surface. Reaction of above is the
net reaction for this process, but the electrons produced from the
oxidation of Ag(s) are not transported over finite distances before
combining with Cl<sub>2</sub>(g) in its reduction to Cl<sup>–</sup>(aq).
No anode or cathode half-cell reactions exist in this system. The
electrons and ions involved in the reaction move only over atomic-scale
distances.


### Fuel cells

Fuel cells are devices to convert chemical energy to electricity and
heat through electrochemical reactions with the fuel and oxygen supplied
to the anode and cathode, respectively. Typical ions migrating through
the electrolyte are $H^{+}$, ${OH}^{-}$, ${CO}_{3}^{2 -}$, and
$O^{2 -}$. In fuel cells with $H^{+}$ as migrating ions, H<sub>2</sub>
molecules are dissociated into $H^{+}$ on the anode, which are combined
with O<sub>2</sub> on the cathode to form H<sub>2</sub>O and release
heat with the half cell and the net cell reactions in their simplest
form as shown by for the anode, for the cathode, and for the net cell,
respectively. Commonly used electrolytes are polymer and phosphoric
acid, and both anode and cathode reactions are facilitated by catalyst,
typically platinum. The thermodynamic limit of power which can be
generated by the fuel cell is represented by

*Eq. 8‑63*
$w = - \mathrm{\Delta}G = - \mathrm{\Delta}_{\ }^{0}G_{cell} + RTln\left( P_{H_{2}}P_{O_{2}}^{1/2} \right)$

For fuel cells with anions as migrating ions, the anions are generated
on the cathode with H<sub>2</sub>O formed and heat generated on the
anode. Their representative cathode reactions are

*Eq. 8‑64* $\frac{1}{2}O_{2} + H_{2}O + 2e^{-}{= 2OH}^{-}$

*Eq. 8‑65* ${\frac{1}{2}O}_{2} + CO_{2} + 2e^{-} = {CO}_{3}^{2 -}$

*Eq. 8‑66* $\frac{1}{2}O_{2} + 2e^{-} = O^{2 -}$.

The anode reaction for is the reaction represented by , operating at low
temperatures and using catalyst for both electrodes. The anode reactions
for and are

*Eq. 8‑67* ${CO}_{3}^{2 -} + H_{2} = H_{2}O + CO_{2} + 2e^{-}$

*Eq. 8‑68* $O^{2 -} + H_{2} = H_{2}O + 2e^{-}$

respectively. To enable the diffusion of ${CO}_{3}^{2 -}$ and $O^{2 -}$
through the cathode and the electrolyte, both fuel cells are operated at
relative high temperatures, with the former typically in molten
carbonate solutions and the latter through solid oxides. Due to the high
operating temperatures, fuels are converted to hydrogen within the fuel
cell itself by a process called internal reforming, removing the need
for precious-metal catalyst and enabling the use of a variety of fuels.
The net cell reaction for all three fuel cells is the same as in the
case of $H^{+}$, represented by .


### Half cell potentials

When electron current flows between electrodes, reactions are occurring
at the electrodes and concentration gradients causing polarization
develop around the electrodes. These gradients result in extraneous
potentials to occur at the electrodes. In such cases cell equilibrium is
not established and measured cell potentials are not those for true
partial equilibrium. If a cell is short-circuited with the electrodes
connected by a conductor, current will flow until the external potential
becomes zero, i.e. ε<sub>ext</sub> = 0, and equilibrium is established
with same conditions as non-electrochemical systems. If an external
potential, ε<sub>ext</sub>, is applied to the cell, chemical reactions
occur until the cell potential balances to ε<sub>ext</sub>, and no
current flows. This potential is called open-circuit voltage (OCV) in
the literature. It is important to realize that OCV includes all
reactions that occur on the electrode surface when the electrode is in
contact with the electrolyte, such as passivation discussed in Chapter .
Partial equilibrium in a cell is achieved when the cell potential is
balanced by an applied external potential. In such partial equilibrium
cases, equilibrium thermodynamic analyses can be used even though the
cell potential is not zero, i.e. ε<sub>cell</sub> ≠ 0. This
differentiates electrochemical systems from other equilibrium systems
discussed previously.

The number of electrons involved in a net cell reaction is important in
relating cell potential and the Gibbs energy change for the cell
reaction. As will be illustrated later in this section, this number
denotes the number of electrons involved in the half-cell reactions that
were added to yield the net cell reaction. The electrical work achieved
by the transport of an electrical charge through a cell potential can be
written as

*Eq. 8‑20* $w = z\ f\ \varepsilon$

where *z* represents the moles of electrons in cell reaction, *f* the
Faraday constant equal to 96,485 J/V/mole-electron, and *ε* the
potential difference, often referred as electromotive force (emf) in the
literature. For a system at constant temperature, pressure, and
composition, this work is the same as the Gibbs energy difference
between the two electrodes, i.e.

*Eq. 8‑21* $- \Delta G = w = zf\varepsilon$

where the negative sign is because the system does work to the
surrounding when the Gibbs energy of the system is decreased. When the
applied external potential is larger than the cell potential, the
surrounding does work to the system, and a common example is the
charging of a battery. Thermodynamic relations discussed in previous
chapters can thus be directly applied to electrochemical systems with
some common equations shown in .

Table ‑: Thermodynamic Equations for Electrochemical Cells

*∆G = -z f ε*

*∆S = - (∂∆G/∂T)P = + z f (∂ε/∂T)P*

*∆H = \[∂(∆G/T)/ ∂(1/T)\]P = - z f \[∂(ε/T)/∂(1/T)\]P = z f \[T(∂ε/∂T)P
– ε\]*

*∆CP = (∂∆H/∂T)P = T z f (∂2ε/∂T2)P*

A half-cell reaction potential cannot be measured directly, only its
potential relative to another half-cell reaction. By convention, a
standard half-cell potential is measured relative to the standard
hydrogen half-cell reduction reaction at 25<sup>o</sup>C (298K) and 1
bar, which has a defined standard potential of zero volts,

*Eq. 8‑22* H<sup>+</sup>(aq, a=1) + e<sup>-</sup> = 1/2 H<sub>2</sub>(g,
1 bar)

with ε<sup>o</sup> (H<sup>+</sup>/H<sub>2</sub>,g) = 0.00 volts. The
standard half-cell reduction reactions of metals at 25<sup>o</sup>C are
for the general reaction

*Eq. 8‑23* M<sup>z+</sup>(aq, a=1) + z e<sup>-</sup> = M(s)

with ε<sup>o</sup> (M<sup>z+</sup>/M) volts. Half-cell reactions with
the most positive standard electrode potentials have a tendency to
spontaneously proceed toward reduction (cathode reactions). Half-cell
reactions with the most negative standard electrode potentials have a
tendency to spontaneously proceed toward oxidation (anode reactions).

Consider, for example, a cell made up of a standard hydrogen electrode
and a standard zinc electrode with ε<sup>o</sup>
(H<sup>+</sup>/H<sub>2</sub>,g) = 0.00 volts and ε<sup>o</sup>
(Zn<sup>2+</sup>/Zn) = -0.762 volts. Thus, the H<sup>+</sup> would tend
to be reduced, and the zinc metal would tend to be oxidized, and the
spontaneous reaction if all species had unit activities would be

*Eq. 8‑24* 2 H<sup>+</sup>(aq, a=1)+ Zn = H<sub>2</sub>(1 bar) +
Zn<sup>2+</sup>(aq, a=1)

with ε<sup>o</sup><sub>cell</sub> = 0.762 volts and
$\mathrm{\Delta}_{\ }^{0}G = - 2*96485*\varepsilon_{cell}^{0}\ $. The
cathode half-cell reaction would be the same as , while the anode
half-cell reaction would be

*Eq. 8‑25* Zn = Zn<sup>2+</sup>(aq, a=1) + 2 e<sup>-</sup>

When ion concentrations and H<sub>2</sub> gas do not all have unit
activities, the Gibbs energy and cell potential of the cell reaction, ,
becomes

*Eq. 8‑26*
$\mathrm{\Delta}G = \mathrm{\Delta}_{\ }^{0}G + RTln\frac{a_{{Zn}^{2 +}}P_{H_{2}}}{\left( a_{H^{+}} \right)^{2}}$

*Eq. 8‑27*
$\varepsilon_{cell} = \varepsilon_{cell}^{0} - \frac{RT}{zf}\ln\frac{a_{{Zn}^{2 +}}P_{H_{2}}}{\left( a_{H^{+}} \right)^{2}}$

The standard reduction potentials of some common metals at
25<sup>o</sup>C are given in .

Table ‑: Standard reduction potentials of some common metals

A cell reaction can be established by different half-cell reactions. For
example, the following reaction can be derived from two different cells

*Eq. 8‑28* $3\ {Fe}^{2 + \ }\  = \ \ 2\ {Fe}^{3 +}\ \  + \ \ Fe(s)$

cell A

*Eq. 8‑29* $3{Fe}^{2 +} + \ 6\ e - \  = \ \ 3Fe(s)$ εo<sub>1</sub> =
-0.440 V

*Eq. 8‑30* $2\ Fe(s)\ \  = \ \ 2\ {Fe}^{3 +}\ \  + \ 6\ e -$
εo<sub>2</sub> = +0.036 V

cell B

*Eq. 8‑31* $2{Fe}^{2 +}\  = \ 2{Fe}^{3 + \ \ } + 2\ e^{-}$
*εo<sub>4</sub> = -0.772 V*

*Eq. 8‑32* ${Fe}^{2 + \ }\  + \ 2\ e - \ \  = \ \ Fe(s)$ *εo<sub>5</sub>
= -0.440 V*

Both give the same net reaction shown by , but with 6 and 2 electrons
and standard cell potentials being εo<sub>cell A</sub> = -0.404 V and
εo<sub>cell B</sub> = -1.212 V, respectively. However, the standard
Gibbs energies of both cells are the same, i.e.

*Eq. 8‑33* $\mathrm{\Delta}_{\ }^{0}G$*<sub>cell A</sub> = -6 f (-0.404)
= + 2.424 f*

*Eq. 8‑34* $\mathrm{\Delta}_{\ }^{0}G$*<sub>cell B</sub> = -2 f (-1.212)
= + 2.424 f*

It is shown that $\mathrm{\Delta}_{\ }^{0}G$ values are independent of
half-cell reactions and depend only on the net reaction because the net
reaction is neutral in electron and balanced in mass.


    display(*md[md.markdown.str.contains("\$")].iloc[:10].markdown.apply(Markdown))

1. extra toc. this should be generated from our content. if we do our job then the two should match.
2. split the rest on headings
3. split out the references which are the last element